__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
# avito_path = 'sevastopol/kvartiry/prodam'

place = 'sevastopol'
# place = 'bahchisaray'
# place = 'sudak'

avito_path = place+'/doma_dachi_kottedzhi/prodam'

raw_data_path = 'data/raw/house/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-15 14:47:22 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-15 14:47:22 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-15 14:47:24 | AdsListParser: downloader init
[INFO    ] 2022-09-15 14:47:24 | AvitoParser: init
[INFO    ] 2022-09-15 14:47:34 | AvitoParser: 3 pages for read
[INFO    ] 2022-09-15 14:47:34 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-15 14:47:38 | AdsListParser: read page 1
[INFO    ] 2022-09-15 14:47:42 | AdsListParser: read page 2
[INFO    ] 2022-09-15 14:47:45 | AdsListParser: read page 3
[INFO    ] 2022-09-15 14:47:45 | AdsListParser: last page detected
[INFO    ] 2022-09-15 14:47:45 | DownloaderSeleniumFirefox: close virtual browser
118
CPU times: user 1.36 s, sys: 99.5 ms, total: 1.46 s
Wall time: 23.6 s


In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

118
118


In [8]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
69,2534680046,Объявление «Дом 90 м² на участке 9 сот.» 8 фотографий,5665000,,"с. Солнечная Долина, Школьная ул., 3А","Продается часть дома 90 м2 с участком В доме три спальни, кухня, санузел, просторный коридор, отопление, канализация, \n\nинтернет, водоснабжение, на участке хоз постройки, есть место для парковки. \n\nТак же на участке есть сад с фруктовыми деревьями. \n\nВ пос",2,2022-09-15 14:47:45.056178
112,2491402063,"Объявление «Дом 325 м² на участке 6,7 сот.» 12 фотографий",35000000,,"с. Миндальное, СНТ Солнечная Долина, Центральный пр-т, 586",Продаю Гостевой дом 7 номеров + цокольный этаж вся техника и мебель новое большой Бассейн.,3,2022-09-15 14:47:45.056178


In [ ]:
df['place'] = place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw_{place}.xlsx',index=False)

In [10]:
# df[['adr']].drop_duplicates()

---

In [9]:
# !ls data/raw/house/avito/

In [8]:
# import pandas as pd

# f = 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx'
# df = pd.read_excel(f)
# df['place'] = 'bahchisaray'
# df.to_excel(f)

----

In [11]:
# !ls data/raw/house/avito

In [12]:
# import pandas as pd

# df = pd.read_excel('data/raw/house/avito/avito_2022-09-15_11-54_raw.xlsx')
# len(df)

In [13]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.float_format', '{:.2f}'.format)

In [14]:
# df.sample(2)

In [15]:
# таунхаус

In [16]:
# df[
# #df['description'].str.lower().str.match(r'.*таунхаус.*')
# #|df['title'].str.lower().str.match(r'.*таунхаус.*')
# # df['description'].str.lower().str.match(r'.*две семьи.*')
# # df['description'].str.lower().str.match(r'.*дуплекс.*')
# ][['title','description']]

In [17]:
# 'is_townhouse'

In [18]:
# df[
#    #df['description'].str.match(r'.*доля.*')
#    df['description'].str.match(r'.*\d\+/\d\+.*')
#    #| df['description'].str.match(r'.*часть\s*дома.*')
# ][['adr','description']]

In [19]:
# df[
# df['description'].str.lower().str.match(r'.*\d+/\d+\s*(часть)?\s*(дом|дол).*')
# |df['description'].str.lower().str.match(r'.*часть\s*(дом|дол).*')
# |df['description'].str.lower().str.match(r'.*продам\s*(дол|часть).*')
# ]

In [20]:
# 1/2 часть дома 
# 1/2 дома 
# 1/2 домовладения

# доля
# 1/2 доли 
# Продам 2/3
# Продам долю
# долю в частном доме 
# Продаю 1/2 
# Продам свою долю 

# часть дома
# часть домовладения
# долю
# доли

In [21]:
# df.iloc[425] # СТ Конус, Фиолентовское ш., 77/77
# 544 по ул. Крепостное шоссе 27/20а 
# 807 три спальни по 15/15/29
# 1373 по адресу Фиолентовское шоссе 87/133 В
# 1446 ес: шоссе Фиолентовское д. 150/39. \
# 1625
# 1634

In [22]:
# AvitoDataCleanerRealtyHouse().transform(df)

In [23]:
# import re
# s = 'Севастополь, Балаклавский муниципальный округ, садовое товарищество Успех, 21'
# s = 'садовое товарищество Тополёк, 10'
# s= 'СТ Яблоня, 2-я ул. Сарандинаки'
# # re.match('.*садов.*',s)
# re.match('.*СТ.*',s)